In [2]:
import sympy as sp

In [4]:
# Define symbols for space and time
x, y, t = sp.symbols('x y t')

# Define functions for the velocity field components v1 and v2
v1 = sp.Function('v1')(x, y, t)
v2 = sp.Function('v2')(x, y, t)
# Velocity vector
v = sp.Matrix([v1, v2])

In [5]:
dv = sp.zeros(2,2)
# Indices: j=0 -> derivative w.r.t x, j=1 -> derivative w.r.t y
#          k=0 -> v1, k=1 -> v2
dv[0,0] = v1.diff(x)  # dee(v1)/∂x
dv[0,1] = v2.diff(x)  # dee(v2)/∂x
dv[1,0] = v1.diff(y)  # dee(v1)/∂y
dv[1,1] = v2.diff(y)  # dee(v2)/∂y

print("Velocity gradients dv")
display(dv)

Velocity gradients dv


Matrix([
[Derivative(v1(x, y, t), x), Derivative(v2(x, y, t), x)],
[Derivative(v1(x, y, t), y), Derivative(v2(x, y, t), y)]])

In [6]:
# Construct the uniaxial viscosity tensor ν_{ijkl}
#    In 2D, i,j,k,l ∈ {0,1}. We can store each component in a dictionary
#    keyed by (i,j,k,l), or we can do direct summation. For clarity, let's
#    build a dictionary "nu_tensor[(i,j,k,l)]".

# Define the two viscosity coefficients
nu2 = sp.Symbol('nu2', real=True, positive=True)
nu3 = sp.Symbol('nu3', real=True, positive=True)

def kronecker(i,j):
    # 2x2 identity, delta[i,i] = 1, else 0
    delta = sp.eye(2)
    return delta[i,j]

# We'll define a helper function that returns ν_{i j k l}
def nu_uniaxial(i, j, k, l):
    """
    Returns the component ν_{i j k l} for a uniaxial 2D fluid
    with viscosity parameters nu2, nu3 and director n = (n_x, n_y).

    Based on the schematic:
      ν_{i j k l} = nu2*(δ_{i j} δ_{k l} + δ_{i l} δ_{j k})
                    + (nu3 - nu2)*[n_i n_j δ_{k l} + ...]
    We'll implement the needed terms carefully.
    """
    # Build the isotropic part: nu2*(δ_{ij} δ_{kl} + δ_{il} δ_{jk})
    iso_part = nu2*(kronecker(i,j)*kronecker(k,l) + kronecker(i,l)*kronecker(j,k))

    # Build the anisotropic correction:
    # (nu3 - nu2)*[ n_i n_j δ_{k l} + n_k n_l δ_{i j} + n_i n_k δ_{j l} + n_j n_l δ_{i k} ]
    # The exact form can vary in references; we'll show a typical combination:
    anis_part = (nu3 - nu2)*(
        n[i]*n[j]*kronecker(k,l) +
        n[k]*n[l]*kronecker(i,j) +
        n[i]*n[k]*kronecker(j,l) +
        n[j]*n[l]*kronecker(i,k)
    )
    return iso_part + anis_part

In [7]:
# Construct the dissipative stress σ^D_{i l} = -ν_{i j k l} ∂_j v_k
sigmaD = sp.zeros(2,2)  # We'll store σ^D_{i,l} in a 2x2 matrix
for i in range(2):      # i = 0..1
    for l in range(2):  # l = 0..1
        expr = 0
        for j in range(2):  # sum over j
            for k in range(2):  # sum over k
                expr += -nu_uniaxial(i, j, k, l)*dv[j,k]
        sigmaD[i,l] = expr

NameError: name 'n' is not defined

In [ ]:
print("Dissipative (viscous) stress sigma^D_{i,l} =")
display(sigmaD)